In [ ]:
#CELL 1 INSTALL
!pip install -q ultralytics clearml pycocotools

In [ ]:
import random
import numpy as np
import torch
os.environ["PYTHONHASHSEED"] = str(42)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
#CELL 2 SET ENV + CLEARML INIT
import os
from clearml import Task
from ultralytics import RTDETR

# Set credentials
os.environ['CLEARML_API_ACCESS_KEY'] = "NBTW2GP5BSEA52690P5UJYYECOJ9IW"
os.environ['CLEARML_API_SECRET_KEY'] = "GBVE-tZB3TAEx3brkeQDvevyNs8CF1Emv8JjCoxg6tVDuMsAWHZvwhs1ornogMdy5Qk"
os.environ['CLEARML_API_HOST']       = "https://api.clear.ml"
os.environ['CLEARML_WEB_HOST']       = "https://app.clear.ml"
os.environ['CLEARML_FILES_HOST']     = "https://files.clear.ml"

experiment_tags = [
    "model_name:retinanet",
    "dataset:voc",
    "platform:kaggle",
    "author:elis",
    "account:jacksonEL2@cardiff.ac.uk",
    # "studio:object-detection"
]

# Initialize ClearML
task = Task.init(
    project_name="CMT318-Object-Detection",
    task_name="RT-DETR Elis Training VOC Kaggle",
    output_uri="./",
    tags=experiment_tags,
    auto_connect_frameworks=True,
    auto_resource_monitoring=True
)
print("✅ ClearML Task ID:", task.id)


In [ ]:
#CELL 3 TRAINING
from ultralytics import RTDETR

# Select model to run
model = RTDETR("rtdetr-l.pt")

# launch a fresh training run
results = model.train(
    data="elis_data.yaml",    # your VOC-style YAML config
    epochs=100,               # total epochs
    imgsz=640,                # input image size
    device=0,                 # use GPU 0
    project="elis_rtdetr",    # output folder prefix
    name="run1",              # subfolder name
    clearml=True              # log all metrics to ClearML
)

# report final metric
print(f"✅ Training finished. Last epoch: {results.epoch}, mAP@0.5={results.metrics.map50:.3f}")

In [ ]:
#CELL 4 RESUME TRAINING (Kaggle timeout)

from ultralytics import RTDETR

# load partially trained checkpoint
model = RTDETR("/kaggle/working/runs/train/elis_rtdetr/run1/weights/last.pt")

# resume training
results = model.train(resume=True)

print(f"✅ Resumed at epoch {results.epoch}")